In [1]:
from pathlib import Path
import numpy as np
import csv
import re
import cv2

In [2]:
# 读取相机参数
def read_calib(calib_file_path):
    with open(calib_file_path, 'r') as calib_file:
        calib = {}
        csv_reader = csv.reader(calib_file, delimiter='=')
        for attr, value in csv_reader:
            calib.setdefault(attr, value)
    return calib

def read_pfm(pfm_file_path):
    with open(pfm_file_path, 'rb') as pfm_file:
        header = pfm_file.readline().decode().rstrip()
        channels = 3 if header == 'PF' else 1
        dim_match = re.match(r'^(\d+)\s(\d+)\s$', pfm_file.readline().decode('utf-8'))
        if dim_match:
            width, height = map(int, dim_match.groups())
        else:
            raise Exception("Malformed PFM header.")

        scale = float(pfm_file.readline().decode().rstrip())
        if scale < 0:
            endian = '<'  # littel endian
            scale = -scale
        else:
            endian = '>'  # big endian

        dispariy = np.fromfile(pfm_file, endian + 'f')

    img = np.reshape(dispariy, newshape=(height, width, channels))
    img = np.flipud(img).astype('uint8')
    print(img.max(), img.min())
    # cv2.imshow("disparity", img)
    return dispariy, [(height, width, channels), scale]

def create_depth_map(pfm_file_path, calib=None):
    dispariy, [shape, scale] = read_pfm(pfm_file_path)

    if calib is None:
        raise Exception("Loss calibration information.")
    else:
        fx = float(calib['cam0'].split(' ')[0].lstrip('['))
        base_line = float(calib['baseline'])
        doffs = float(calib['doffs'])
        # scale factor is used here
        # d = bf/(d+ doffs)         doffs就是(x_or-x_ol) 两个相机主点在各自图像坐标系x方向上的坐标差
        depth_map = fx * base_line / (dispariy / scale + doffs)
        depth_map = np.reshape(depth_map, newshape=shape)
        depth_map = np.flipud(depth_map).astype('uint8')
        return depth_map

# def show(img, win_name='image'):
#     if img is None:
#         raise Exception("Can't display an empty image.")
#     else:
#         # cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
#         cv2.imshow(win_name, img)
#         cv2.waitKey()
#         cv2.destroyWindow(win_name)



In [3]:
import cv2
import numpy as np
import torch
from utils import to_pixel_samples, make_coord, warp_coord
from torchvision import transforms


# 加载左图像和视差图
left_img = transforms.ToTensor()(cv2.imread('demo\Pipes-perfect\im0.png')).unsqueeze(0)
right_img = transforms.ToTensor()(cv2.imread('demo\Pipes-perfect\im1.png')).unsqueeze(0)
print(left_img.shape)
_,c,h,w = left_img.shape
hr_coord, hrl_rgb = to_pixel_samples(left_img.contiguous())
_, hrr_rgb = to_pixel_samples(right_img.contiguous())
sample_lst = np.random.choice(
                len(hr_coord), 2304, replace=False)
hr_coord = hr_coord.unsqueeze(0)
hrl_rgb = hrl_rgb.unsqueeze(0)
hrr_rgb = hrr_rgb.unsqueeze(0)
print(hr_coord.shape, hrl_rgb.shape)

torch.Size([1, 3, 1924, 2960])
torch.Size([1, 5695040, 2]) torch.Size([1, 5695040, 3])


In [16]:
pfm_file_dir = Path('demo/Pipes-perfect')
calib_file_path = pfm_file_dir.joinpath('calib.txt')
disp_left = pfm_file_dir.joinpath('disp1.pfm')
disparity, [shape, scale] = read_pfm(disp_left)
print(disparity)
disparity = np.reshape(disparity, newshape=shape)
disparity = np.flipud(disparity)
print(disparity.shape)
# calibration information
calib = read_calib(calib_file_path)
# create depth map
depth_map_left = create_depth_map(disp_left, calib)
# show(depth_map_left, "depth_map")

255 0
[149.93364 149.91214 149.88821 ... 116.0108  115.87364 115.84978]
(1924, 2960, 1)
255 0


C:\Users\Administrator\AppData\Local\Temp\ipykernel_46592\3867768275.py:30: RuntimeWarning: invalid value encountered in cast
  img = np.flipud(img).astype('uint8')


In [8]:
# 创建x和y映射
height, width, _ = disparity.shape
x_map = np.float32(np.tile(np.linspace(0, width-1, width), (height, 1)))
y_map = np.float32(np.tile(np.linspace(0, height-1, height), (width, 1)).T)

# 考虑到视差，更新x映射
x_map = x_map - disparity[:,:,0]

# 使用remap函数进行warp
warped_img = cv2.remap(left_img, x_map, y_map, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

# 保存结果
cv2.imwrite('vis/warped.png', warped_img)

error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'remap'
> Overload resolution failed:
>  - src is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'src'


In [17]:

# disparity = cv2.imread('vis/out_disp.jpg', cv2.IMREAD_GRAYSCALE)
# disparity = np.expand_dims(disparity, -1)
print(disparity.shape)
disparity_new = transforms.ToTensor()(disparity.copy())
hrl_d = disparity_new.view(1, -1).permute(1, 0)
hrl_d = hrl_d.unsqueeze(0)

# left_warp = warp_coord(hr_coord, hrl_d, right_img)
right_warp = warp_coord(hr_coord, hrl_d, left_img, mode='l2r')
# left_warp = left_warp.permute(0, 2, 1).contiguous()

# diff = left_warp - hrl_rgb
diff = right_warp - hrr_rgb

print(diff)
# vis
# left_warp = left_warp * 255.
# left_warp = left_warp.view(1, h, w, 3).squeeze(0).permute(2, 0, 1).contiguous().numpy().astype(np.uint8).transpose(1,2,0)
# cv2.imwrite('vis/warped_new.png', left_warp)

right_warp = right_warp * 255.
right_warp = right_warp.view(1, h, w, 3).squeeze(0).permute(2, 0, 1).contiguous().numpy().astype(np.uint8).transpose(1,2,0)
cv2.imwrite('vis/warped_new1.png', right_warp)

diff = diff.view(1, h, w, 3).squeeze(0).permute(2, 0, 1).contiguous()
print(diff.mean())






(1924, 2960, 1)
tensor([[[-0.0078, -0.0157,  0.0157],
         [-0.0039, -0.0157,  0.0039],
         [-0.0078, -0.0196, -0.0039],
         ...,
         [-0.0353, -0.0235, -0.0235],
         [-0.0353, -0.0275, -0.0275],
         [-0.0353, -0.0275, -0.0275]]])
tensor(0.0268)


In [7]:
from utils import warp
import torch
img_tensor = torch.from_numpy(np.expand_dims(left_img, 0).transpose(0, 3, 1, 2)).float()
print(img_tensor.shape)
print(img_tensor.type())
disparity_tensor = torch.from_numpy(disparity.copy()).unsqueeze(0).permute(0, 3, 1, 2).float()
print(disparity_tensor.type())
print(disparity_tensor)
warped_img_new = warp(img_tensor, disparity_tensor, mode='r2l')
warped_img_new = warped_img_new.view(1,img_tensor.shape[2],img_tensor.shape[3],-1).permute(0,3,1,2).squeeze(0).numpy().astype(np.uint8).transpose(1,2,0)
print(warped_img_new.shape)
# 保存结果
cv2.imwrite('vis/warped_new.png', warped_img_new)

ValueError: axes don't match array

In [8]:
import torch
a = torch.rand([1,2,3,4])
print(a)
a2 = a.contiguous().reshape(3,4,2)
print(a2)

tensor([[[[0.0495, 0.1418, 0.3764, 0.2652],
          [0.4315, 0.6114, 0.6444, 0.8426],
          [0.7846, 0.6737, 0.1217, 0.9667]],

         [[0.0181, 0.0375, 0.1108, 0.6531],
          [0.4091, 0.0390, 0.0898, 0.2533],
          [0.3682, 0.1509, 0.7292, 0.1315]]]])
tensor([[[0.0495, 0.1418],
         [0.3764, 0.2652],
         [0.4315, 0.6114],
         [0.6444, 0.8426]],

        [[0.7846, 0.6737],
         [0.1217, 0.9667],
         [0.0181, 0.0375],
         [0.1108, 0.6531]],

        [[0.4091, 0.0390],
         [0.0898, 0.2533],
         [0.3682, 0.1509],
         [0.7292, 0.1315]]])
